## Build base Docker image (if not already built) 

In [4]:
%%bash
docker build -t mlflow-docker-example -f bike-rental-regression/Dockerfile . 

#1 [internal] load build definition from Dockerfile
#1 sha256:a92a3c0eb5f2b81ad04f5f888ca51d5c5451050e601d95d6743cd6c6e792818d
#1 transferring dockerfile: 121B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:78fa54d501a98b0f549aad8cdfa59820f924aad699b0b9ad6b0f0d531561b6da
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/continuumio/miniconda3:4.10.3
#3 sha256:f692b03fa4ad9171171f333a701a320801f86a5a153c52ecb1003a5c4c2ae3ce
#3 ...

#4 [auth] continuumio/miniconda3:pull token for registry-1.docker.io
#4 sha256:5aa71fa7b66ff2060b6761f35695c1cb52e7cd95eaecd23637375560251b2975
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/continuumio/miniconda3:4.10.3
#3 sha256:f692b03fa4ad9171171f333a701a320801f86a5a153c52ecb1003a5c4c2ae3ce
#3 DONE 3.4s

#5 [1/2] FROM docker.io/continuumio/miniconda3:4.10.3@sha256:a137c7da98c8680467490e15ac3c54e25db77495be1737076b053a6790ad6082
#5 sha256:303ec1e2d8b90ac6a39c06473d2c8520b602780f3a5a452cdee2d369

## Train a model 

In [9]:
%%bash
mlflow run bike-rental-regression -P train_year=2011 -P train_month=1 -P test_year=2011 -P test_month=3 -P dataset_path=/data/train_bike.csv

2021/11/06 23:58:38 INFO mlflow.projects.docker: === Building docker image bike-rental-regression:9270a06 ===
2021/11/06 23:58:40 INFO mlflow.projects.utils: === Created directory /tmp/tmpxdlzskjf for downloading remote URIs passed to arguments of type 'path' ===
2021/11/06 23:58:40 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/andrea/demo-ATSS/mlruns:/mlflow/tmp/mlruns -v /home/andrea/demo-ATSS/mlruns/0/c258b637e0a64ffe979248f475d4c94b/artifacts:/home/andrea/demo-ATSS/mlruns/0/c258b637e0a64ffe979248f475d4c94b/artifacts -v /home/andrea/demo-ATSS/data:/data -e MLFLOW_RUN_ID=c258b637e0a64ffe979248f475d4c94b -e MLFLOW_TRACKING_URI=file:///mlflow/tmp/mlruns -e MLFLOW_EXPERIMENT_ID=0 bike-rental-regression:9270a06 python train.py 2011 1 2011 3 /data/train_bike.csv' in run with ID 'c258b637e0a64ffe979248f475d4c94b' === 
/opt/conda/lib/python3.9/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Inte

## Package the model in a Docker image and register it in the local registry

In [10]:
%%bash
mlflow models build-docker --model-uri runs:/c258b637e0a64ffe979248f475d4c94b/model --name "my-image-name"

/tmp/tmpjcpizwnn/
/tmp/tmpjcpizwnn/model_dir
/tmp/tmpjcpizwnn/model_dir/model
/tmp/tmpjcpizwnn/model_dir/model/model.pkl
/tmp/tmpjcpizwnn/model_dir/model/conda.yaml
/tmp/tmpjcpizwnn/model_dir/model/MLmodel
/tmp/tmpjcpizwnn/model_dir/model/requirements.txt
/tmp/tmpjcpizwnn/model_dir/model/input_example.json
/tmp/tmpjcpizwnn/Dockerfile


2021/11/06 23:58:58 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2021/11/06 23:58:58 INFO mlflow.models.docker_utils: Building docker image with name my-image-name
#1 [internal] load build definition from Dockerfile
#1 sha256:fce7d8a58319781ec0965487a7f04561a9f31fa7242886e45c2cf2fa2ddf128f
#1 transferring dockerfile: 1.58kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:1ceab46ae4c0ce9c98808b79da6cd3f19a2b85acc496494ea19e1f13f2054286
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:d5f6ad098564dfa0e5b5bccfbeaacc4eaa8bc13eb61c81cec010e23191b2b2cc
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 DONE 2.6s

#5 [ 1/12] FROM docker.io/l

## Start the docker container

Run this command in a separate shell, because it needs to run in the background. It will be killed with ^C.

```
docker run -p 5001:8080 "my-image-name"
```

## Try to get a prediction

In [13]:
import requests
import json
import pandas as pd

dataset = pd.read_csv('data/train_bike.csv', header=0,  sep=',',
                      parse_dates=['datetime'], index_col='datetime').drop(
    columns=['casual', 'registered']
)
test_year = 2011
test_month = 3
test: pd.DataFrame = dataset.loc[
    lambda df: (df.index.year == test_year) & (df.index.month == test_month)
]
def time_features(input_df):
    return input_df.assign(
        month = lambda df: df.index.month,
        day = lambda df: df.index.day,
        hour = lambda df: df.index.hour,
    )
test.pipe(time_features).drop(columns='count').head().to_json(orient='split', index=False)

predictions = requests.post(
    'http://127.0.0.1:5001/invocations',
    json=json.loads(
        test.pipe(time_features)
        .drop(columns='count')
        .to_json(orient='split', index=False)
    )
).json()

from sklearn.metrics import r2_score

r2_score(test.pipe(time_features)['count'].values, predictions)

0.6026563896049335